# Divide the tweets to state-month files

In [1]:
ls /n/holyscratch01/cga/dkakkar/gov_project/results/

2017/  2018/  2019/  census/  tweet_states/


In [2]:
import pandas as pd
import os
from tqdm.notebook import tqdm,trange
import geopandas as gpd
from pandarallel import pandarallel
import us
# import tqdm
# pandarallel.initialize(progress_bar=True)
tqdm.pandas()

def get_data(row):
    file_name = row["in_file_name"]
    data = pd.read_csv(input_path+ file_name, delimiter = '\t', quotechar  = "\"",dtype={"fips":str,"GPS":str})
    data['tweet_date'] = pd.to_datetime(data['tweet_date'], errors='coerce',unit="s")
    return data
    
def split_ms(input_path,output_path,year):
    print("reading..")
    pd_inputFiles = pd.DataFrame(os.listdir(input_path),columns=["in_file_name"])
    pd_inputFiles["month"] = pd_inputFiles["in_file_name"].map(lambda x: list(map(int,x.split(".")[0].split("_")))[1])
    # pd_inputFiles["data"] =  pd_inputFiles.parallel_apply(get_data,axis=1)
    pd_inputFiles["data"] =  pd_inputFiles.progress_apply(get_data,axis=1)
    
    print("writing..")
    for k,data in pd_inputFiles.groupby("month"):
        if data["data"].values.shape[0] == 0:
            continue
        data_ = pd.concat(data["data"].values,ignore_index=True)
        for i,item in data_.groupby("state"):
            fine_path = output_path + "%d_%d_%s.parquet"%(year,k,i)
            if os.path.exists(fine_path):
                continue
            try:
                item.to_parquet(fine_path)
            except Exception as e:
                # print(e)
                pass

# df.apply(func)
def save_tweets(x,alread_set):
    file_name = x.file_name

    s = us.states.lookup(file_name.split("_")[2])
    # print(s.abbr)
 
    cenus_shp = gpd.read_file(path + file_name)
    cenus_shp = cenus_shp.to_crs("EPSG:4326")
    
    for i in trange(1,13):
        out_filename = "%d_%d_%s.csv"%(year,i,s.abbr)
        if out_filename in alread_set:
            continue
        

        f = output_path + "%d_%d_%s.parquet"%(year,i,s.abbr)
        # print(f)
        
        # cenus_shp = gpd.read_file(path + file_name)
        # cenus_shp = cenus_shp.to_crs("EPSG:4326")
        try:
            df = pd.read_parquet(f)
        except Exception as e:
            # print(e,f)
            continue
        df = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.longitude,df.latitude),crs="EPSG:4326")
        join_inner_df = df.sjoin(cenus_shp, how="inner")
        join_inner_df= join_inner_df[['message_id', 'tweet_date', 'tweet_text', 'tweet_lang', 'place',
               'user_id', 'user_name', 'user_location', 'followers', 'friends',
               'latitude', 'longitude', 'GPS', 'spatialerror', 'fips', 'county',
               'state',  'GEOID20']]
        join_inner_df.to_csv(outpath + out_filename,encoding = "utf-8-sig")
        
        # break
    return 1

            
path = "/n/holyscratch01/cga/xiaokang/Census_Blocks/"
#  2017 splot files done!  
year = 2019
# for year in [2018,2019]:
print(year)
input_path = "/n/holyscratch01/cga/dkakkar/gov_project/results/%d/"%year
output_path = '/n/holyscratch01/cga/xiaokang/gov_project/results/tweet_states/%d/parquet/'%year
outpath = '/n/holyscratch01/cga/xiaokang/gov_project/results/census/%d/'%year

os.makedirs(output_path,exist_ok=True)
os.makedirs(outpath,exist_ok=True)

# split_ms(input_path,output_path,year)       

alread_set =  set(os.listdir(outpath))
states_list = pd.Series(os.listdir(path),name="file_name").to_frame()
temp=states_list.progress_apply(lambda x:save_tweets(x,alread_set),axis=1)
            

2019


  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]